In [11]:
# -*- coding: utf-8 -*-
"""lda-himank.ipynb
Original file is located at
    https://colab.research.google.com/drive/12zx-jsZowxlpoN5yMB7CbRrZWVJVBfMu
"""

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
import re
import numpy as np
import pandas as pd
from pprint import pprint
import os
from pathlib import Path
import json
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet as wn
import csv
import random
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.test.utils import datapath
import spacy
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import json
import random


[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
with open('sampled_te_docs_10k.json', 'r') as outfile:  
    test_docs = json.load(outfile)
with open('sampled_tr_docs_10k.json', 'r') as outfile:  
    train_docs = json.load(outfile)

In [64]:
with open('sampled_tr_docs_10k_not_shuffled.json', 'r') as outfile:  
    train_docs_unshuffled = json.load(outfile)
with open('sampled_tr_docs_10k_trustworthy.json', 'r') as outfile:  
    train_docs_trustworthy = json.load(outfile)

In [13]:
def clean(doc):
    deacc = gensim.utils.simple_preprocess(doc, deacc=True, min_len=4)
    stop_free = [i for i in deacc if i not in stop]
    normalized = [lemma.lemmatize(word) for word in stop_free]
    return normalized

stop = set(stopwords.words('english'))
lemma = WordNetLemmatizer()

In [14]:
## LOAD dict
dictionary = corpora.Dictionary.load_from_text('dict_10k')

In [15]:
with open('doc_term_matrix.json', 'r') as outfile:
    doc_term_matrix = json.load(outfile)
with open('test_doc_term_matrix.json', 'r') as outfile:
    test_doc_term_matrix = json.load(outfile)


In [16]:
###### LOAD MODEL
Lda = gensim.models.wrappers.LdaMallet
ldamodel_10k = Lda.load("model_10k_5000_alpha")

In [17]:
# len(test_docs)
len(train_docs)

70000

In [18]:
DOC_NUM_FROM_TESTING = 16000  # range is 0 - 184706
DOC_NUM_FROM_TRAINING = 500   # range is 0 - 70000

In [19]:
print(test_docs[DOC_NUM_FROM_TESTING])

As Americans begin the Memorial Day weekend, we remember those who have given their lives to defend the freedoms and way of life that we enjoy. The Heritage Foundation’s James Carafano writes in The Sacramento Bee that as we honor them, we must also “do our utmost not to add to their ranks”:

Cold gray monuments, brassy parades, majestic flyovers – they are all remembrances of those who died in the service of the nation. They are all part of our Memorial Day. No day speaks more about American patriotism than the day we thank those who gave their lives in the fight for freedom. Yet, no ceremony, no solemnity can ever replace those we have lost . . . So while on this day we honor sacrifice, we have a job the rest of the year as well: reminding our leaders in Washington to ensure that the troops who defend us have what they need to do the job – and come back to us. There is no better way to recognize the valor of those who serve, and demonstrate care and respect for their families, than t

In [20]:
tops = ldamodel_10k[test_doc_term_matrix[DOC_NUM_FROM_TESTING]]
tops = sorted(tops, key=lambda a:a[1], reverse=True)
for top in tops[:10]:
    print(top[1], ldamodel_10k.show_topic(top[0]))
    print("\n")
    

0.269611183791489 [('military', 0.050022350411886164), ('force', 0.031943350575079646), ('defense', 0.020636880308222822), ('army', 0.01681602418101705), ('soldier', 0.011086513832421579), ('missile', 0.010100257561889354), ('troop', 0.009614224795475993), ('weapon', 0.009380077623335249), ('navy', 0.008950807807743886), ('operation', 0.008929521701185637)]


0.2030686363031811 [('american', 0.02050535392062583), ('world', 0.01931544371089909), ('people', 0.019300586716226), ('government', 0.018875136414223816), ('america', 0.017171984570335714), ('state', 0.013565786772412452), ('country', 0.011732973884104636), ('nation', 0.011507417692249512), ('united', 0.008934456342045835), ('power', 0.00876967876476245)]


0.13758735018774282 [('protest', 0.02107634585819354), ('event', 0.016034035594281407), ('people', 0.015471886139183095), ('protester', 0.013321238602632733), ('group', 0.012320442224238011), ('flag', 0.01166034248529681), ('march', 0.010438093291257299), ('rally', 0.009752441

In [65]:
doc_term_matrix_unshuffled = [dictionary.doc2bow(doc) for doc in [clean(d) for d in train_docs_unshuffled]]
doc_term_matrix_trustworthy = [dictionary.doc2bow(doc) for doc in [clean(d) for d in train_docs_trustworthy]]

In [ ]:
print(train_docs[DOC_NUM_FROM_TRAINING])

In [68]:
print(train_docs_unshuffled[5000])

In truth, there is only one powerplant a Ford Mustang should be purchased with and that's the V-8. If an Internet forum is to be believed, it sounds like Ford somewhat agrees – the automaker is doing away with its entry-level V-6, leaving the Ecoboost four-cylinder and the torquey V-8. Mustang6g forum user XKRJAG claims some snooping through the vehicle ordering system uncovered that the option numbers for the 300-horsepower V-6 -engined Mustangs for the 2018 model year no longer work, which could mean the entry-level power plant has been discontinued. If this is true, it would mean for that for the non-Shelby Mustangs, customers would only be able to choose from the 310-horsepower turbocharged four-cylinder motor and the 435-horsepower V-8. Both are quality options and, at least for us at The Drive , tears for the nixed sixer would not be shed.
Additionally, XKRJAG noticed there are new performance packs for the GT trim level, which could hint that a track-friendly, Chevy Camaro SS 1L

In [115]:
print(train_docs_unshuffled[100])

In just the last week, we've seen new chatter about the possibility that Donald Trump could put the House in play for Democrats, as non-partisan analysts, giddy liberals, and even some anguished Republicans have started wondering whether Trump could pull off such a remarkable feat. But while it's certainly not impossible to imagine, any such conclusions are very premature. There's still far too much we don't know—and won't know—unless and until Trump is the nominee (and of course, he may not be). Democrats have certainly bet wrong before on the effects a supposedly "unacceptable" Republican standard-bearer might have for his party.

But one thing we do know does not augur well for Democrats: Because a House takeover has seemed so remote this cycle, the party has struggled to land strong candidates in a large enough number of potentially competitive districts to even contemplate overcoming its 30-seat deficit. The prospect of a Trumpocalypse has only started to feel real quite recently,

In [116]:
tops = ldamodel_10k[doc_term_matrix_unshuffled[100]]
tops = sorted(tops, key=lambda a:a[1], reverse=True)
for top in tops[:10]:
    print(top[1], ldamodel_10k.show_topic(top[0]))
    print("\n")

0.4932227433128682 [('republican', 0.041565425076621476), ('candidate', 0.020566998923157633), ('party', 0.02026327415302206), ('campaign', 0.018737747466659304), ('democrat', 0.018113040837175912), ('conservative', 0.015351906563216168), ('romney', 0.011576055443576221), ('democratic', 0.011444901565563133), ('mccain', 0.009305022503244333), ('election', 0.00901510340447856)]


0.1387479528261895 [('issue', 0.010244002700313087), ('policy', 0.007932612659223678), ('time', 0.006680794288944131), ('problem', 0.006625866660524222), ('make', 0.005489219124028357), ('effort', 0.005403283963435918), ('action', 0.005152565917583752), ('interest', 0.005145478481658602), ('decision', 0.005013474987552691), ('change', 0.004852235820255538)]


0.12098641767144316 [('fact', 0.0106188420960851), ('time', 0.00631593861714186), ('point', 0.00604891446507978), ('make', 0.005253291481384602), ('word', 0.005048391315720638), ('question', 0.0047715581131746445), ('real', 0.004736681489231842), ('claim',

In [22]:
##SHOULD THIS BE doc_term_matrix, you had test_doc_term_matrix??
tops = ldamodel_10k[doc_term_matrix[DOC_NUM_FROM_TRAINING]]
tops = sorted(tops, key=lambda a:a[1], reverse=True)
for top in tops[:10]:
    print(top[1], ldamodel_10k.show_topic(top[0]))
    print("\n")

0.31787755521408834 [('military', 0.050022350411886164), ('force', 0.031943350575079646), ('defense', 0.020636880308222822), ('army', 0.01681602418101705), ('soldier', 0.011086513832421579), ('missile', 0.010100257561889354), ('troop', 0.009614224795475993), ('weapon', 0.009380077623335249), ('navy', 0.008950807807743886), ('operation', 0.008929521701185637)]


0.206582285139536 [('attack', 0.03996745321399512), ('terrorist', 0.030124375217947228), ('terrorism', 0.016068813204696036), ('security', 0.013437173079158433), ('terror', 0.013423224456584911), ('group', 0.012302685109845403), ('killed', 0.01049401371614553), ('state', 0.01032663024526328), ('bomb', 0.00823898640009299), ('islamic', 0.00806230384749506)]


0.1517611207159785 [('israel', 0.07646042066546745), ('iran', 0.04999547779597825), ('israeli', 0.040157171713111374), ('palestinian', 0.02898732777939684), ('iranian', 0.018983207549065913), ('state', 0.014435880171642766), ('jewish', 0.012074284738063894), ('arab', 0.01206

# Below This Is Katie Sketching Out Comparison Stuff

In [24]:
K = ldamodel_10k.num_topics
print(K)

100


In [76]:
print(type(ldamodel_10k[doc_term_matrix[0]]))

<class 'list'>


In [118]:
# Unshuffled Matrix => 0-20k -> trustworthy; 20k-70k -> untrustworthy
THRESH = 0.1
all_trusted_tops = ldamodel_10k[doc_term_matrix_unshuffled[:20000]]
all_trusted_tops_lines = list(map(lambda x: [round(v, 3) if v > THRESH else 0 for t, v in x], all_trusted_tops))

In [ ]:
######## Feel free to look at the output
######## Uncomment whatever you want to see

# UNSHUFFLED_DOC = 100

# # prints doc
# print(train_docs_unshuffled[UNSHUFFLED_DOC])

# # prints english topics
# topics = ldamodel_10k[doc_term_matrix_unshuffled[UNSHUFFLED_DOC]]
# tops = sorted(topics, key=lambda a:a[1], reverse=True)
# for top in tops[:10]:
#     print(top[1], ldamodel_10k.show_topic(top[0]))
#     print("\n")
    
# # prints wihtout rounding
# print(all_trusted_tops[UNSHUFFLED_DOC])

# # prints with rounding as is written as a row of csv
# print(all_trusted_tops_lines[UNSHUFFLED_DOC])

In [89]:
with open('Matrices/topic_doc_matrix.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(all_trusted_tops_lines)

In [138]:
import gensim.matutils as mt

In [ ]:
term_topic_matrix = ldamodel_10k.get_topics()
K = 100
topic_comps = np.zeros((K,K))
ind = list(range(term_topic_matrix.shape[1]))
for i1 in range(K):
    print('!!!' + str(i1))
    a = list(term_topic_matrix[i1, :])
    #print('A')
    a = list(zip(ind, a))
    #print('B')
    #a = [(1,4)]
    #print(a)
    for j1 in range(i1, K):
        #print(j1)
        b = list(term_topic_matrix[j1, :])
        #print('a')
        b = list(zip(ind, b))
        #print('b')
        sim = mt.cossim(a, b)
        #print('c')
        topic_comps[i1, j1] = sim
        #print('d')
        topic_comps[j1, i1] = sim  
        #print('e')

In [ ]:
np.savetxt("topic_sims.csv", topic_comps, delimiter=",")

In [ ]:
def get_docs(top, matches):
    tot_length = 0
    docs = []
    scores = []
    while(tot_length < 5000 and len(docs) < 3):
        doc = matches[top, 'Match_' + str(len(docs) + 1)]
        docs.append(doc)
        scores.append(matches[top, 'Val_' + str(len(docs) + 1)])
        tot_length += len(doc.split()) 
    return docs, scores

In [ ]:
#Do sampling for documents to make summary for
nums = [random.randing(0, 9999) for i in range(5*30)]
type_nums = [10000*(i%30+2) for i in range(30*5)]
sampled_doc_nums = tuple(map(sum, zip(nums, type_nums)))
# Unshuffled Matrix => 0-20k -> trustworthy; 20k-70k -> untrustworthy

full_documents = # load this some how    sampled_tr_docs_10k_not_shuffled.json
matches = #load this somehow    topics_match_indices.csv

THRESH = 0.1
all_untrusted_tops = ldamodel_10k[doc_term_matrix_unshuffled[sampled_doc_nums]]
all_untrusted_tops_lines = list(map(lambda x: [round(v, 3) if v > THRESH else 0 for t, v in x], all_trusted_tops))

SUM_THRESH = .15
doc_matches_matrix = []
for doc in all_untrusted_tops_lines:
    matched_documents = [full_documents[doc]]
    for top in doc:
        if(top < SUM_THRESH):
            continue
        topic_docs, scores = get_docs(top, matches)
        matched_documents.append({'documents': topic_docs, 'scores': scores})
    with open('matches/' + str(doc) +'.json', 'w') as fout:
        json.dump(matched_documents, fout)

In [ ]:
###Summarizing done in a separate script